## Import Libraries

In [1]:
import pandas as pd
import panel as pn

pn.extension(sizing_mode='stretch_width') # setting the sizing mode for panels/widgets
# pn.extension('tabulator') # we don't need to load specific extension for deploying the notebooks as an app.

## Read Data

In [2]:
df = pd.read_pickle('sampledata.pkl')
df['Comment'] = '' # Adding a new column for commenting feature

## Create Tabulator

In [ ]:
# create a tabulator for interactively vieweing the data, filtering and so on
# make rows selectable with a checkbox
# set all columns non-editable except the Comment column
# add header filters
# add additional configuration
df_tabulator = pn.widgets.Tabulator(df,
                                    show_index=False, 
                                    page_size=15,
                                    sizing_mode='stretch_width', 
                                    layout='fit_data', 
                                    selectable='checkbox', 
                                    editors={
                                        'TimeGenerated': None,
                                        'MachineName': None,
                                        'AppName': None,
                                        'Comment': 'input'
                                    }, 
                                    header_filters={
                                        'MachineName': {'type': 'input', 'func': 'like', 'placeholder': 'Enter Machine'},
                                        'AppName': {'type': 'input', 'func': 'like', 'placeholder': 'Enter App'},
                                        'AppCategory': {'type': 'input', 'func': 'like', 'placeholder': 'Enter Category'},
                                        'TotalMBytes': {'type': 'number', 'func': '>=', 'placeholder': 'Min MB'}
                                    },
                                    configuration={'resizable': True,
                                                   'headerSort': True,
                                                   'selectablePersistence': True
                                                  }
                                   )

## Create Widgets

In [ ]:
# create a date range slider based on the TimeGenerated values of `df`
# set start and end values
# set default values when the slider is called
# set step in milliseconds (it is applied when moving the slider)
dt_range_slider = pn.widgets.DatetimeRangeSlider(sizing_mode='stretch_width',
    name='Date Range Slider',
    start=df['TimeGenerated'].min(), end=df['TimeGenerated'].max(),
    value=(df['TimeGenerated'].min(), df['TimeGenerated'].min() + pd.Timedelta(days=1)),
    step = 1800000
)

In [ ]:
machines = sorted(df['MachineName'].unique())
machine_select = pn.widgets.MultiSelect(options=machines, value=machines[:1], name='Machine Filter', sizing_mode=None)

In [ ]:
# Bind widgets to tabulator as filters for correspoinding columns

df_tabulator.add_filter(dt_range_slider, 'TimeGenerated')
df_tabulator.add_filter(machine_select, 'MachineName')

In [ ]:
# put all the pieces together
my_dashboard = pn.Column('Interactive Data Analysis', pn.Row(dt_range_slider, sizing_mode='stretch_width'),
                         pn.Row('Select Machine\(s\)'), pn.Row(machine_select, df_tabulator))

## Deploy the app

In [ ]:
# Make the final view servable
pn.config.sizing_mode = "stretch_both"
pn.config.template = 'material'
my_dashboard.servable(title = 'My Awesome Interactive Dashboard');